In [ ]:
import pandas as pd
import zipfile
import kaggle

## Loading Data

Downloading a dataset from kaggle

In [ ]:
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

  0% 0.00/165k [00:00<?, ?B/s]
100% 165k/165k [00:00<00:00, 63.6MB/s]


Extracting data from zip file

In [ ]:
with zipfile.ZipFile('london-bike-sharing-dataset.zip', 'r') as file:
  file.extractall()

In [ ]:
df = pd.read_csv('/content/london_merged.csv')
df.shape

(17414, 10)

In [ ]:
df.head()

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0


## Initial Data Analysis

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [ ]:
df.isnull().sum()

timestamp       0
cnt             0
t1              0
t2              0
hum             0
wind_speed      0
weather_code    0
is_holiday      0
is_weekend      0
season          0
dtype: int64

In [ ]:
df.duplicated().sum()

0

In [ ]:
df.describe()

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
count,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000,17414.000000
mean,1143.101642,12.468091,11.520836,72.324954,15.913063,2.722752,0.022051,0.285403,1.492075
std,1085.108068,5.571818,6.615145,14.313186,7.894570,2.341163,0.146854,0.451619,1.118911
min,0.000000,-1.500000,-6.000000,20.500000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,257.000000,8.000000,6.000000,63.000000,10.000000,1.000000,0.000000,0.000000,0.000000
50%,844.000000,12.500000,12.500000,74.500000,15.000000,2.000000,0.000000,0.000000,1.000000
75%,1671.750000,16.000000,16.000000,83.000000,20.500000,3.000000,0.000000,1.000000,2.000000
max,7860.000000,34.000000,34.000000,100.000000,56.500000,26.000000,1.000000,1.000000,3.000000


## Exploratory Data Analysis

In [ ]:
columns = {
    'timestamp': 'time',
    'cnt': 'count',
    't1': 'temp_real_C',
    't2': 'temp_feels_like_C',
    'hum': 'humidity_percent',
    'wind_speed': 'wind_speed_kph',
    'weather_code': 'weather'
}

season_dict = {
    '0.0': 'spring',
    '1.0': 'summer',
    '2.0': 'autumn',
    '3.0': 'winter'
}

weather_dict = {
    '1.0': 'Clear',
    '2.0': 'Scattered clouds',
    '3.0': 'Broken clouds',
    '4.0': 'Cloudy',
    '7.0': 'Rain',
    '10.0': 'Rain with thunderstorm',
    '26.0': 'Snowfall'
}

Rename columns

In [ ]:
df.rename(columns, axis=1, inplace=True)

Mapping seasonal and wether columns

In [ ]:
df.season = df.season.astype('str').map(season_dict)
df.weather = df.weather.astype('str').map(weather_dict)

Normalizing humidity percent to [0, 1]

In [ ]:
df.humidity_percent = df.humidity_percent / 100

Converting 0 for No and 1 for Yes in is_holiday and is_weekend

In [ ]:
df.is_holiday = df.is_holiday.astype('str').map({'0.0': 'No', '1.0': 'Yes'})
df.is_weekend = df.is_weekend.astype('str').map({'0.0': 'No', '1.0': 'Yes'})

In [ ]:
df.head()

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,No,Yes,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,No,Yes,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,No,Yes,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,No,Yes,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,No,Yes,winter


In [ ]:
df.to_excel('bikes_data.xlsx', sheet_name='Data')